# Python --> try with stochastic target to emulate real life

In [1]:
import importlib
import numpy as np
import time
import string

In [2]:
import dna_v3

In [3]:
from dna_v3 import DNA # To get this to work, needed to put 'if name == main' at bottom of dna.py 

In [4]:
importlib.reload(dna_v3)

<module 'dna_v3' from '/home/mjmrose/workspace/sbox-mjmrose/Bids/genAlgo/dna_v3.py'>

# Testing

In [19]:
tgt = "To be or not to be."
pop_sz = 8
mut_rate = 0.3
fit_exp = 2
pop = Population(tgt, mut_rate, pop_sz, fit_exp, 2)

In [33]:
pop.evolve()

World Record: 0.10526315789473684
Best: T$Ft=S5r*z4;FO!Lxo 
Average: 0.046052631578947366


In [40]:
pop.mating_pool

[(5, 0.3333333333333333),
 (6, 0.3333333333333333),
 (7, 0.3333333333333333),
 (0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0)]

In [41]:
for i in range(len(pop.population)):
    print(pop.population[i].genes, pop.population[i].fitness)

['d', '6', 'o', '9', 'W', 'Z', '1', '<', 'n', '1', 'c', '#', 'Y', '#', '5', 'I', 'D', 'd', '6'] 0.0
['\\', 's', 'z', ']', '4', 'N', '8', 'h', 'k', '/', 'f', '#', 'L', ' ', 'W', 'p', 'A', ',', 'u'] 0.0
['C', 'j', 'g', 'o', 'w', '&', 'f', 'B', 'Z', 'F', 'V', 'Y', 'K', 'h', '9', '6', 'A', 'Q', '!'] 0.0
['V', 'D', '=', 'z', 'O', ']', 'o', 'H', '$', ')', '`', '=', 'C', '-', ' ', 'i', ')', '-', 'H'] 0
['V', 'i', 'r', "'", 's', ']', 'y', 'H', '$', 'P', '$', 'x', '?', '-', '6', 'v', 'x', '-', 'j'] 0
['V', 'i', 'r', "'", 'O', ']', 'o', 'H', '$', ')', '$', '!', 'C', '-', '>', 'i', ')', '-', 'm'] 0.0027700831024930744
['V', 'i', 'r', "'", 'O', 'P', 'o', 'H', '$', 'R', '$', '!', 'C', '-', '>', 'i', ')', '-', 'm'] 0.0027700831024930744
['V', 'i', 'r', "'", 'O', 'p', 'o', 'H', '$', 'R', '(', 'X', '?', '0', '6', 'v', ')', '-', 'm'] 0.0027700831024930744


# Class Definition

In [5]:
# A class to describe a population of virtual organisms
# In this case, each organism is just an instance of a DNA object

class Population:
    def __init__(self, tgt, mut_rate, pop_sz, fit_exp, num_new_mems, replace=True, midpt=False, stoc_tgt=False, verbose=False, debug=False):

        self.population = [] 
        self.mating_pool = [] 
        self.generations = 0
        self.finished = False 
        self.tgt = tgt
        self.mut_rate = mut_rate
        self.perfect_score = 1
        self.best = ""
        self.fitness_sum = 0
        self.fit_exp = fit_exp # Raise fitness to this power to increase (if > 1) probability of higher fitness members breeding
        self.num_new_mems = num_new_mems # Number of new mems to add to the population or to replace in existing population
        self.replace_bool = replace # True: replace 'num_new_mems' mems of pop w/ lowest fitness w/ new children // False: new children will be appended to existing population
        self.midpt_bool = midpt # True: Choose point to split mems being bred and combine // False: probabilistically select gene from one of mems being bred based on fitness
        self.stoc_tgt_bool = stoc_tgt # True: tgt will change every 100 generations // False: tgt remains stagnant
        self.verbose = verbose
        self.debug = debug

        for i in range(pop_sz):
            self.population.append( DNA( len(self.tgt), verbose=self.verbose ) )

            
    def calc_fitness(self):
        '''
        Calculates fitness for every member of the population, exponentiates the fitness and calculates population sum
        '''
        if self.debug: print("start of calc_fitness")
        self.fitness_sum = 0
    
        for i in range( len(self.population) ):
           
            self.population[i].calc_fitness(self.tgt)
            
            self.population[i].fitness = self.population[i].fitness**self.fit_exp
            
            self.fitness_sum += self.population[i].fitness

            
    def gen_mating_pool(self):
        '''
        Generates mating pool as sorted list of tuples (pop_idx, exponentiated_fitness) w/ highest scoring mems first
        '''
        if self.debug: print("start of gen_mating_pool")
        self.mating_pool = []
    
        for i in range( len(self.population) ): 
    
            # Appending (idx, normalized fitness) for each mem of pop
            self.mating_pool.append( (i, self.population[i].fitness / max(self.fitness_sum, 1e-4) ) )
    
        # Sorting by fitness score in descending order
        self.mating_pool.sort(reverse=True, key = lambda x : x[1])
        
    
    def pick_mem_from_mating_pool(self):
        '''
        Selects a member (mem) from the mating pool to participate in crossover.

        Steps for selection:
            1). Draw random number b/w 0-1 (val)
            2). Subtract normalized fitness of 1st mem of mating pool (highest fitness) from val
            3). If val is now negative, mem of pop corresponding to first mem of mating pool is selected for crossover.
            4). If val is still positive, move to 2nd mem of mating pool and repeat until val is negative
        '''
        if self.debug: print("Start of pick_mem_from_mating_pool")
            
        val = np.random.random()
        
        if self.verbose: print(f"val: {val}")
        if self.verbose: print(f"mating pool: {self.mating_pool}")
            
        for i in range( len(self.mating_pool) ):
            val -= self.mating_pool[i][1]
            if val < 0:
                break
        if self.verbose: print(f"idx of mating pool: {self.mating_pool[i][0]}")
        return self.mating_pool[i][0]
    
    def gen_new_pop(self):
        '''
        Generates new members (mems) of population by probabilistically mating existing mems of the population.
        Mems of the population w/ higher fitness are more likely to be selected for mating.
        '''
        if self.debug: print("Start of gen_new_pop")
        children = []
        
        for i in range( self.num_new_mems ): 
                
            idx1 = self.pick_mem_from_mating_pool()
            idx2 = self.pick_mem_from_mating_pool()
            
            #THIS WAS NEW --> would want with low mutation rate
            # Sampling a random (top 10 fitness) member if idx1 == idx2, w/ 50% probability
            if (idx1 == idx2) & (np.random.random() > 0.5): 
                rand_hi_idx = np.random.randint( min(10, len(self.population)) )
                idx2 = self.mating_pool[rand_hi_idx][0]
                if self.verbose: print("idx1 == idx2 dealt with")
            
            partnerA = self.population[idx1]
            partnerB = self.population[idx2]
            
            if self.verbose: print(f"PartnerA: {partnerA.get_phrase()}")
            if self.verbose: print(f"PartnerB: {partnerB.get_phrase()}")
    
            child = partnerA.crossover(partnerB, midpt_bool=self.midpt_bool)
            child.mutate(self.mut_rate)
            if self.verbose: print(f"Child: {child.get_phrase()}")
                
            children.append(child)

        if self.replace_bool:    
            for i in range(len(children)):
                replace_idx = self.mating_pool[len(self.mating_pool) - i - 1][0]
                self.population[replace_idx] = children[i] # Overwrites self.population[replace_idx].fitness w/ 0
        else:
            self.population.extend(children)

        self.generations += 1

    def get_best(self):
        return self.best

    def evaluate(self):
        '''
        Computes the current "most fit" member of the population and whether the perfect score has been achieved.
        '''
        world_record = 0
        idx = 0
        for i in range(len(self.population)): 
            if self.population[i].fitness > world_record:
                idx = i
                world_record = self.population[i].fitness
                
        print(f"World Record: {world_record**(1/self.fit_exp)}")

        self.best = self.population[idx].get_phrase()
        if world_record**(1/self.fit_exp) == self.perfect_score:
            self.finished = True
            
    def get_new_char(self):
        all_possible_chars = string.printable[:-10] + " "
        idx = np.random.randint(len(all_possible_chars))
        return all_possible_chars[idx]

    def is_finished(self):
        return self.finished

    def get_generations(self):
        return self.generations

    def get_average_fitness(self):
        total = 0
        for i in range( len( self.population ) ):
            total += self.population[i].fitness**(1/self.fit_exp)
        return total / len(self.population)
    
    def evolve(self):
    
        # Calculate fitness for each mem of pop, take fitness**fit_exp and calc fitness sum
        self.calc_fitness()

        # Generate mating pool array by sorting normalized fitness values
        self.gen_mating_pool()

        # Generate new population mems by crossover b/w existing mems of mating pool
        # Either replace existing mems or add new mems to pop
        self.gen_new_pop()

        # Compute most fit mem of pop and determine if finished
        self.evaluate()

        print(f"Best: {self.get_best()}")
        print(f"Average: {self.get_average_fitness()}")

        # If we found the target phrase, stop
        if self.is_finished():
            print("We did it :)")
            print(f"Result: {self.get_best()}")
            print(f"Num gens: {self.get_generations()}")
        
        # Updating target if target declared as stochastic
        if (self.stoc_tgt_bool) & (self.get_generations() % 100 == 0):
            idx = np.random.randint(len(self.tgt))
            new_val = self.get_new_char()
            print(f"\nReplacing {self.tgt[idx]} at idx {idx}: {self.tgt[idx]} w/ {new_val}\n")
            new_tgt = []
            for i in range(len(self.tgt)):
                if i != idx:
                    new_tgt.append(self.tgt[i])
                else:
                    new_tgt.append(new_val)
            self.tgt = ''.join(new_tgt)
            print(f"New self.tgt: {self.tgt}\n")
            
    
    

### Initializing a population

In [34]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.2
fit_exp = 2
pop = Population(tgt, mut_rate, pop_sz, fit_exp, 400, stoc_tgt = True)

# Testing speed of convergence w/ gene selection based on fitness

In [35]:
start = time.time()
for i in range(2000):
    pop.evolve()
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.10526315789473684
Best: ym*uvMor/R!AIk>MwiM
Average: 0.01136842105263157
World Record: 0.21052631578947367
Best: e# bk ;N;^&AQ\FcNeL
Average: 0.02336842105263156
World Record: 0.2631578947368421
Best: )(*5%Mor/=ot#9o4mig
Average: 0.034947368421052574
World Record: 0.2631578947368421
Best: >=`bk ;r;f&A 5iuNeL
Average: 0.043157894736842096
World Record: 0.42105263157894735
Best: [=Qbk mr;foA 5osbeL
Average: 0.04884210526315792
World Record: 0.42105263157894735
Best: [=Qbk mr;foA 5osbeL
Average: 0.05673684210526315
World Record: 0.47368421052631576
Best: [=Qbk mr;foA 5o beR
Average: 0.06210526315789463
World Record: 0.47368421052631576
Best: [=Qbk mr;foA 5o beR
Average: 0.06936842105263154
World Record: 0.47368421052631576
Best: [=Qbk mr;foA 5o beR
Average: 0.07515789473684217
World Record: 0.5263157894736842
Best: [+rbk o9 not (  bDz
Average: 0.08021052631578941
World Record: 0.5789473684210527
Best: E=.be &r;fot ,o beR
Average: 0.08568421052631572
World Record: 0.5789473

# Testing midpt

In [10]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.3
fitness_exp = 2
pop = Population(tgt, mut_rate, pop_sz, fitness_exp)

TypeError: __init__() missing 1 required positional argument: 'num_new_mems'

In [ ]:
start = time.time()
for i in range(1000):
    draw(pop, 100)
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.10526315789473684
Best: ([&x"6C8 0ZSHWq tGP
Average: 0.01026315789473683
World Record: 0.15789473684210525
Best: /HBFC*<U xHf2\3 b_$
Average: 0.01661654135338342
World Record: 0.15789473684210525
Best: /HBFC*<U xHf2\3 b_$
Average: 0.022828947368421018
World Record: 0.21052631578947367
Best: Yv'bq&trgF4uppoabAS
Average: 0.030292397660818735
World Record: 0.21052631578947367
Best: Yv'bq&trgF4uppoabAS
Average: 0.036105263157894835
World Record: 0.2631578947368421
Best: WQ'bYA0l K4trtsDb(*
Average: 0.04124401913875615
World Record: 0.2631578947368421
Best: WQ'bYA0l K4trtsDb(*
Average: 0.0467543859649125
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.0517408906882594
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.05687969924812078
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.06063157894736904
World Record: 0.3157894736842105
Best: ;FZ$5BMr nN!=to 5#i
Average: 0.06434210526315867
World Record: 0.3684210526

World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1401947368420947
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14062011464303267
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1408204334365218
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14092999489012717
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14124493927124412
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1414586466165303
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14169811320753592
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14187899655680125
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14214912280700612
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14239497827135497
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.14272248803826598
World Record: 0.47368421

World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15502590673573458
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1551112316874494
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15527935222670386
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1554162191192098
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15555971146137773
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1556698564593133
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.1557762496693828
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15593157894735157
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15598586017280333
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15611255862426662
World Record: 0.47368421052631576
Best: 0C be0Zr nH,ftoNFd.
Average: 0.15626393570130545
World Record: 0.473684210

# Debugging

In [ ]:
draw(pop, 5, True)

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())



In [ ]:
pop.mating_pool

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())


In [ ]:

pop.mating_pool